<a href="https://colab.research.google.com/github/lbk209/topic_modeling/blob/main/tm_wine_reviews_params_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⛏️ Setting

In [5]:
%%capture
!pip install bertopic accelerate adjustText

In [4]:
# testing
#!rm bertopic_custom_run.py bertopic_custom_util.py

fatal: not a git repository (or any of the parent directories): .git


In [1]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

import pandas as pd
import numpy as np

import plotly.express as px

In [1]:
from bertopic_custom_run import bertopic_batch
from bertopic_custom_util import utils, read_csv, print_with_line_feed, param_study

In [2]:
count_wine = lambda x: x.groupby(by=['wid','wine']).id.count().rename('count').reset_index(1)

# 🗂️ Data

In [3]:
file = 'wine_reviews'
path_data = 'sample_data'
path_src = '/content/drive/MyDrive/Colab\ Notebooks/'

In [10]:
!unzip {path_src}/{file}.zip -d {path_data}

Archive:  /content/drive/MyDrive/Colab Notebooks//wine_reviews.zip
  inflating: sample_data/wine_reviews_240124.csv  
  inflating: sample_data/wine_reviews_240207.csv  
  inflating: sample_data/wine_reviews_240212.csv  
  inflating: sample_data/wine_reviews_240213a.csv  
  inflating: sample_data/wine_reviews_240213b.csv  


In [11]:
df_reviews = read_csv(file, path_data, parse_dates=['date'])
df_reviews = df_reviews.rename_axis('id').reset_index()
df_reviews.head()

,id,wid,wine,date,review,source,lang,review_transl
0,0,40,Arione Moscato d'Asti,2023-11-06,pera pesca nota tropicale minerali,vivino,it,pear peach tropical note minerals
1,1,40,Arione Moscato d'Asti,2023-10-17,Ottimo,vivino,it,Optimal
2,2,40,Arione Moscato d'Asti,2023-10-09,"조금 산뜻한 향, 처음 마시고 조금 있는 바디감, Saracco보다 조금 덜 달게 ...",vivino,ko,"A slightly refreshing scent, a bit of body whe..."
3,3,40,Arione Moscato d'Asti,2023-10-08,"Afrutado, delicioso.",vivino,es,"Fruity, delicious."
4,4,40,Arione Moscato d'Asti,2023-07-21,honey apple peach citrus Slight fizziness ligh...,vivino,en,honey apple peach citrus Slight fizziness ligh...


## Review data

*reviews not translated*

In [12]:
#df_reviews.loc[df_reviews.lang.str.contains('ERROR')]

cond = df_reviews.review_transl.isna()
df_reviews.loc[cond]

,id,wid,wine,date,review,source,lang,review_transl
5816,5816,7,Kendall-Jackson Vintner's Reserve Cabernet Sau...,2023-03-18,:-),vivino,<-- ERROR -->,NaN
6090,6090,9,Frontera Cabernet Sauvignon,2022-05-21,😡,vivino,<-- ERROR -->,NaN
6136,6136,9,Frontera Cabernet Sauvignon,2022-01-24,🤌🏻,vivino,<-- ERROR -->,NaN
6423,6423,12,The 7th Generation - G7 Chardonnay,2022-01-31,🌵🍏🥝🧈,vivino,<-- ERROR -->,NaN
6627,6627,13,San Pedro Gato Negro Chardonnay,2022-04-27,… …. ….,vivino,<-- ERROR -->,NaN
7012,7012,18,Montes Montes Alpha Cabernet Sauvignon,2022-12-19,♡♡♡,vivino,<-- ERROR -->,NaN
7128,7128,18,Montes Montes Alpha Cabernet Sauvignon,2021-12-24,.,vivino,<-- ERROR -->,NaN


In [13]:
df_reviews = df_reviews.loc[~cond]

In [14]:
#df_reviews.groupby(['wid', 'wine']).review_transl.count()
df_reviews.groupby(['wid', 'wine']).id.count().tail()

wid  wine                                       
45   Cono Sur Single Vineyard Syrah                  31
46   Cono Sur Single Vineyard Cabernet Sauvignon     32
47   Cono Sur Single Vineyard Pinot Noir             40
48   Cono Sur Single Vineyard Carmenere              13
49   Cono Sur Single Vineyard Chardonnay            238
Name: id, dtype: int64

In [15]:
#df_reviews.loc[df_reviews.wid < 19].groupby(['wid', 'wine']).id.count().sum()
df_reviews.id.count()

7132

### Wine group

*check wines of small numbers of reviews*

In [ ]:
cnt = 50
df = count_wine(df_reviews)
list_wid = df.loc[df['count']>cnt].index

df_reviews_tmp = df_reviews.loc[df_reviews.wid.isin(list_wid)]
count_wine(df_reviews_tmp)

,wine,count
wid,,
0,Casillero del Diablo Cabernet Sauvignon (Reserva),472
1,Yellow Tail Cabernet Sauvignon,136
2,Roche Mazet Cuvée Spéciale Cabernet Sauvignon,61
3,San Pedro Gato Negro Cabernet Sauvignon,879
6,Casillero del Diablo Chardonnay (Reserva),122
7,Kendall-Jackson Vintner's Reserve Cabernet Sau...,121
8,Viña Santa Helena Reservado Cabernet Sauvignon',51
9,Frontera Cabernet Sauvignon,207
10,Long Barn Chardonnay,142


the favourite red wine group

In [16]:
list_wid = [7, 18, 19, 20, 24, 25, 28, 42, 43, 44]

df_reviews_tmp = df_reviews.loc[df_reviews.wid.isin(list_wid)]
count_wine(df_reviews_tmp)

,wine,count
wid,,
7,Kendall-Jackson Vintner's Reserve Cabernet Sau...,121
18,Montes Montes Alpha Cabernet Sauvignon,220
19,Montes Montes Alpha Merlot,60
20,Montes Montes Alpha Syrah,81
24,Mollydooker The Boxer Shiraz,276
25,William Hill North Coast Cabernet Sauvignon,51
28,Bread & Butter Pinot Noir,1236
42,Two Hands Angels' Share Shiraz,82
43,Two Hands Gnarly Dudes Shiraz,102


moscato d'asti group

In [ ]:
df = count_wine(df_reviews)
list_wid = df.loc[df.wine.str.lower().str.contains('moscato')].index

df_reviews_tmp = df_reviews.loc[df_reviews.wid.isin(list_wid)]
count_wine(df_reviews_tmp)

,wine,count
wid,,
30,Vietti Moscato d'Asti,41
32,Michele Chiarlo Moscato d'Asti Nivole,482
33,Canti Moscato d'Asti,71
34,Vallebelbo Moscato d'Asti,32
35,Balbi Soprani Moscato d'Asti,31
36,Gancia Moscato d'Asti N.V.,160
37,Tosti Moscato d'Asti,31
38,G.D. Vajra Moscato d'Asti,52
39,Castello del Poggio Moscato d'Asti,80


#### Confirm the group

In [17]:
df_reviews = df_reviews_tmp

### Documents for modeling

In [18]:
# DO NOT CHANGE the (document) id as it is index to topics of topic model (topic_model.topics_)
docs = df_reviews.review_transl.tolist()
print(f'num of reviews: {df_reviews.id.count()}')
#docs[:3]

num of reviews: 2311


# 🗨️ Topic modeling

*Check docs defined in Data*

In [19]:
count_wine(df_reviews)

,wine,count
wid,,
7,Kendall-Jackson Vintner's Reserve Cabernet Sau...,121
18,Montes Montes Alpha Cabernet Sauvignon,220
19,Montes Montes Alpha Merlot,60
20,Montes Montes Alpha Syrah,81
24,Mollydooker The Boxer Shiraz,276
25,William Hill North Coast Cabernet Sauvignon,51
28,Bread & Butter Pinot Noir,1236
42,Two Hands Angels' Share Shiraz,82
43,Two Hands Gnarly Dudes Shiraz,102


*Embeddings*

In [20]:
st_id = 'all-MiniLM-L12-v2'
embedding_model = SentenceTransformer(st_id)
embeddings = embedding_model.encode(docs, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/73 [00:00<?, ?it/s]

*Parameters*

In [21]:
param_set = {
    'min_df': 0.0,
    'max_df': .6,
    'n_components': 100, #100, 50, 30
    'n_neighbors': 15, #5, 20, 30
    'min_dist': 0,
    'min_cluster_size': 20, #50, 10
    'min_samples': 10, #20, 10
}

param_set.update({
    'embedding_model': embedding_model,
    'embeddings': embeddings,
    'reduced_embeddings': True, # Pre-reduce embeddings for visualization purposes
    'random_state': 42,
})

*run*

In [22]:
tm_post = bertopic_batch(docs,
                         calculate_probabilities=True,
                         hdbscan_model=None,
                         **param_set)

df_info = tm_post.print_topic_info()

# output of get_topic_info
df_info.head(5)

num of topics: 24
outliers: 0.250


In [24]:
n = 10
_ = tm_post.print_custom_labels(min_count=n)

0 good wine; wine good; nice wine; great wine; wine great; wines; wine; red wine; wine light; sauvignon 
1 pepper blackberry oak; plum blackberry oak; blackberry plum oak; blackberry oak pepper; vanilla oak blackberry; 
  chocolate plum blackberry; oak blackberry vanilla; oak blackberry chocolate; chocolate blackberry plum; oak blackberry 
  oak 
2 good pinot noir; nice pinot noir; pinot noir good; best pinot noir; pinot noir great; pinot noir; pinot noir best; 
  pinot noir really; pinot noirs; californian pinot noir 
3 butter vanilla oak; vanilla butter oak; vanilla oak butter; oak cherry butter; cherry vanilla oak; oak vanilla cherry; 
  vanilla oak cherry; oak butter cherry; oak butter; oak vanilla 
4 flavorful pinot; tasting pinot; really good pinot; pinot strawberry; pinot vanilla; pinot good; pinot fantastic; pinot 
  beautiful; pinot nice; nice pinot 
5 plum tobacco chocolate; plum blackberry; dark cherry; dark fruit; red fruit; black cherry; berries; black fruit; plum 
  tobac

In [25]:
tm_visual = tm_post.visualize()

In [28]:
tm_visual.visualize_documents(docs, list_tid=None,
                              width=1000, height=800
                              )

In [29]:
_ = tm_post.print_custom_labels([0,1,2,3])

0 good wine; wine good; nice wine; great wine; wine great; wines; wine; red wine; wine light; sauvignon 
1 pepper blackberry oak; plum blackberry oak; blackberry plum oak; blackberry oak pepper; vanilla oak blackberry; 
  chocolate plum blackberry; oak blackberry vanilla; oak blackberry chocolate; chocolate blackberry plum; oak blackberry 
  oak 
2 good pinot noir; nice pinot noir; pinot noir good; best pinot noir; pinot noir great; pinot noir; pinot noir best; 
  pinot noir really; pinot noirs; californian pinot noir 
3 butter vanilla oak; vanilla butter oak; vanilla oak butter; oak cherry butter; cherry vanilla oak; oak vanilla cherry; 
  vanilla oak cherry; oak butter cherry; oak butter; oak vanilla 


### Topic similarity

In [38]:
tm_post.check_similarity().head(10)

,topic1,topic2,distance
0,1_blackberry_plum_oak_chocolate,5_medium_nose_acidity_black,0.821844
1,15_red fruit_red_vanilla_strawberry,1_blackberry_plum_oak_chocolate,0.814633
2,15_red fruit_red_vanilla_strawberry,5_medium_nose_acidity_black,0.808946
3,23_disgusting_taste light_light taste_tastes,9_scent_aroma_fruity scent_little,0.807736
4,5_medium_nose_acidity_black,9_scent_aroma_fruity scent_little,0.807663
5,15_red fruit_red_vanilla_strawberry,3_oak_butter_vanilla_vanilla oak,0.805556
6,19_spicy_value_light_price,6_drink_easy drink_light_drinking,0.794565
7,19_spicy_value_light_price,9_scent_aroma_fruity scent_little,0.793185
8,19_spicy_value_light_price,23_disgusting_taste light_light taste_tastes,0.785560
9,2_noir_pinot noir_pinot_vanilla,4_pinot_price_light_pinots,0.773511


In [37]:
from sentence_transformers.util import pytorch_cos_sim

(tm_post
 .check_similarity(
     custom_labels=True,
     min_distance=0.7,
     pytorch_cos_sim=pytorch_cos_sim, embedding_model=embedding_model,
 )
 .sort_values('c/label sim', ascending=False)
 .head(10))

Calculating the similarity of custom label pairs for which the topic similarity exceeds 0.7...


,topic1,topic2,distance,c/label sim
5,15 vanilla red fruit; strawberry red fruit; va...,3 butter vanilla oak; vanilla butter oak; vani...,0.805556,0.831104
1,1 pepper blackberry oak; plum blackberry oak; ...,15 vanilla red fruit; strawberry red fruit; va...,0.814633,0.770548
0,1 pepper blackberry oak; plum blackberry oak; ...,5 plum tobacco chocolate; plum blackberry; dar...,0.821844,0.763421
10,1 pepper blackberry oak; plum blackberry oak; ...,3 butter vanilla oak; vanilla butter oak; vani...,0.759266,0.760632
8,19 light wonderful flavors; wonderful flavors ...,23 light taste decent; taste decent; light usu...,0.785560,0.740660
2,15 vanilla red fruit; strawberry red fruit; va...,5 plum tobacco chocolate; plum blackberry; dar...,0.808946,0.714865
14,11 quite buttery smooth; quite buttery; butter...,7 smooth; smooth smooth; light smooth; smooth ...,0.754860,0.672993
27,18 mmmmmm yummmmyyyyy tasty; mmmmmm; delicious...,19 light wonderful flavors; wonderful flavors ...,0.706478,0.639765
17,21 fantastically good; fantastically good eleg...,22 bad; bad really liked; bad really; imo bad;...,0.751567,0.624886
9,2 good pinot noir; nice pinot noir; pinot noir...,4 flavorful pinot; tasting pinot; really good ...,0.773511,0.623676


In [42]:
tid = [15,3]
_ = tm_post.print_custom_labels(tid)

3 butter vanilla oak; vanilla butter oak; vanilla oak butter; oak cherry butter; cherry vanilla oak; oak vanilla cherry; 
  vanilla oak cherry; oak butter cherry; oak butter; oak vanilla 
15 vanilla red fruit; strawberry red fruit; vanilla oak strawberry; cherry strawberry raspberry; butter strawberry 
  raspberry; cherry butter strawberry; oak strawberry raspberry; oak butter strawberry; strawberry raspberry red; red 
  fruit vanilla 


In [43]:
df_info.loc[df_info.Topic.isin(tid)]

,Topic,Count,Name,CustomName,Representation,KeyBERT,Representative_Docs
4,3,111,3_oak_butter_vanilla_vanilla oak,3 butter vanilla oak; vanilla butter oak; vani...,"[oak, butter, vanilla, vanilla oak, cherry]","[butter vanilla oak, vanilla butter oak, vanil...","[Butter, vanilla, oak, vanilla oak cherry butt..."
16,15,36,15_red fruit_red_vanilla_strawberry,15 vanilla red fruit; strawberry red fruit; va...,"[red fruit, red, vanilla, strawberry, red frui...","[vanilla red fruit, strawberry red fruit, vani...","[Vanilla oak strawberry raspberry red fruit, v..."


*merge topics*

In [44]:
topics_to_merge = [tid]
tm_post.topic_model.merge_topics(docs, topics_to_merge)

# update custom labels
tm_post.set_custom_labels()

In [46]:
df_info = topic_model.get_topic_info()

#df_info.loc[df_info.Topic.isin(tid)]
df_info.head(5)

,Topic,Count,Name,CustomName,Representation,KeyBERT,Representative_Docs
0,-1,578,-1_vanilla_oak_light_fruit,-1 flavors; oak vanilla; strawberry; red fruit...,"[vanilla, oak, light, fruit, red]","[flavors, oak vanilla, strawberry, red fruit, ...",[vanilla oak butter strawberry red fruit smoke...
1,0,476,0_wine_red_vanilla_oak,0 good wine; wine good; nice wine; great wine;...,"[wine, red, vanilla, oak, light]","[good wine, wine good, nice wine, great wine, ...","[Good wine @, Good wine, ^^ Good wine ^^]"
2,1,147,1_oak_vanilla_butter_vanilla oak,1 vanilla butter oak; butter vanilla oak; vani...,"[oak, vanilla, butter, vanilla oak, cherry]","[vanilla butter oak, butter vanilla oak, vanil...",[vanilla oak cherry butter strawberry raspberr...
3,2,123,2_blackberry_plum_oak_chocolate,2 pepper blackberry oak; plum blackberry oak; ...,"[blackberry, plum, oak, chocolate, pepper]","[pepper blackberry oak, plum blackberry oak, p...","[vanilla oak blackberry pepper plum, oak black..."
4,3,117,3_noir_pinot noir_pinot_vanilla,3 good pinot noir; nice pinot noir; pinot noir...,"[noir, pinot noir, pinot, vanilla, light]","[good pinot noir, nice pinot noir, pinot noir ...","[Most delicious Pinot Noir, good for pinot noi..."


### Topic per class

In [48]:
classes_all = df_reviews.wine.tolist()

Review of topics before merging topics

- Topic 0 is about good wine: Though bread & butter pinot garners more overall praise, kendall-jackson cab boasts a higher ratio of favorable comments relative to its total comments.
- Topic 1, focusing on oak, chocolate, plum, and blackberry flavors, is the least representative of bread & butter pinot, william hill cab and montes syrah.
- Topics 2 & 3 are about pinot noir: the majority of comments in the topics are from the only pinot wine, bread & butter.
- Topic 4 seems about the 2 montes wines, but william hill cab shares the topic.
- Topic 5 is about easy driniking, the most representative of bread & butter pinot and william hill cab.
- Topic 6, centered on favorite wines, is most emblematic of Gnarly Dudes and William Hill, despite the limited number of comments on the topic.
- Topic 7, featuring buttery vanilla oak notes, starkly contrasts with Topic 1, as it exclusively pertains to the Bread & Butter Pinot.
- Topic 8 discusses cherry and acidity, with Montes Syrah being the wine most associated with this characteristic, intriguingly surpassing even the Bread & Butter Pinot.
- Topic 9, centered on fruity scents, is most closely associated with Montes Merlot.

In [50]:
tm_visual.topics_per_class_all(docs, classes_all)

In [51]:
tid = list(range(10))
_ = tm_post.print_custom_labels(tid)

0 good wine; wine good; nice wine; great wine; wine great; wines; wine; red wine; wine light; sauvignon 
1 vanilla butter oak; butter vanilla oak; vanilla oak butter; oak cherry butter; cherry vanilla oak; oak vanilla cherry; 
  vanilla oak cherry; oak butter cherry; fruit vanilla oak; strawberry vanilla oak 
2 pepper blackberry oak; plum blackberry oak; pepper oak blackberry; blackberry plum oak; blackberry oak pepper; vanilla 
  oak blackberry; chocolate plum blackberry; oak blackberry vanilla; oak blackberry chocolate; chocolate blackberry plum 
3 good pinot noir; nice pinot noir; pinot noir good; best pinot noir; pinot noir great; pinot noir; pinot noir best; 
  pinot noir really; pinot noirs; californian pinot noir 
4 flavorful pinot; tasting pinot; really good pinot; pinot strawberry; pinot vanilla; pinot good; pinot fantastic; pinot 
  beautiful; pinot nice; nice pinot 
5 plum tobacco chocolate; plum blackberry; dark cherry; dark fruit; red fruit; black cherry; berries; black fr

In [55]:
a = count_wine(df_reviews).wine.to_dict()
a

{7: "Kendall-Jackson Vintner's Reserve Cabernet Sauvignon",
 18: 'Montes Montes Alpha Cabernet Sauvignon',
 19: 'Montes Montes Alpha Merlot',
 20: 'Montes Montes Alpha Syrah',
 24: 'Mollydooker The Boxer Shiraz',
 25: 'William Hill North Coast Cabernet Sauvignon',
 28: 'Bread & Butter Pinot Noir',
 42: "Two Hands Angels' Share Shiraz",
 43: 'Two Hands Gnarly Dudes Shiraz',
 44: 'Two Hands Sexy Beast Cabernet Sauvignon'}

visualize the selected wines

In [56]:
group = [7,18,20,25,28,43]
group = [v for k,v in a.items() if k in group]

tm_visual.topics_per_class_all(docs, classes_all, group=group)

# 🎚️ **Parameter Study**

## Set params

In [62]:
params_study = {
    # When CountVectorizer building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.
    # If float, the parameter represents a proportion of documents, integer absolute counts
    #'min_df': [1, 10], # error when 10 even if default max_df is 1.0
    'min_df': [0.001, 0.01],

    'max_df': [0.5, 1.0],

    # If you are interested in (density based) clustering, or other machine learning techniques,
    # it can be beneficial to pick a larger embedding dimension (say 10, or 50) closer to the the dimension
    # of the underlying manifold on which your data lies.
    'n_components': [10, 20, 50],

    # how UMAP balances local versus global structure in the data.
    # low values of n_neighbors will force UMAP to concentrate on very local structure (potentially to the detriment of the big picture)
    'n_neighbors': [10, 20, 50],

    # the minimum distance apart that points are allowed to be in the low dimensional representation.
    # This means that low values of min_dist will result in clumpier embeddings.
    # This can be useful if you are interested in clustering, or in finer topological structure.
    # Larger values of min_dist will prevent UMAP from packing points together and will focus on the preservation of the broad topological structure instead.
    'min_dist': [0, 0.05, 0.1],

    # set it to the smallest size grouping that you wish to consider a cluster.
    # It can have slightly non-obvious effects with min_samples
    'min_cluster_size': [20, 50, 100],

    # The implementation defaults this value (if it is unspecified) to whatever min_cluster_size is set to.
    # The larger the value of min_samples you provide, the more conservative the clustering –
    # more points will be declared as noise, and clusters will be restricted to progressively more dense area
    # UPDATE as the values are ratio to min_cluster_size
    'min_samples': [0.1, 0.5, 1.]
}

In [63]:
# common params
params_base = {
    'ngram_range': (1, 3),
    'top_n_words': 5,
    'random_state': 42,
    'reduced_embeddings': False,
    'calculate_probabilities': True,
    'hdbscan_model': None,
}

*Reviews to use in param search*

In [67]:
#max_wid = 18
#docs = df_reviews.loc[df_reviews.wid <= max_wid].review_transl.tolist()
len(docs)

2311

In [65]:
# Embedding model: 12-layer, 384-hidden
st_id = 'all-MiniLM-L12-v2'
embedding_model = SentenceTransformer(st_id)
embeddings = embedding_model.encode(docs, show_progress_bar=False)

Batches:   0%|          | 0/73 [00:00<?, ?it/s]

*Update base params with embedding model*

In [68]:
params_base.update({
    'embedding_model': embedding_model,
    'embeddings': embeddings
})

Product param sets

In [69]:
from itertools import product

param_names = params_study.keys()

param_values = params_study.values()
param_values = list(product(*param_values))

df_params = pd.DataFrame(param_values, columns=param_names)

try:
    df_params.min_samples = df_params.min_samples.mul(df_params.min_cluster_size).astype(int)
except:
    print('No min_samples')

df_params = df_params.reset_index(drop=True)
df_params.head(5)

,min_df,max_df,n_components,n_neighbors,min_dist,min_cluster_size,min_samples
0,0.001,0.5,10,10,0.0,20,2
1,0.001,0.5,10,10,0.0,20,10
2,0.001,0.5,10,10,0.0,20,20
3,0.001,0.5,10,10,0.0,50,5
4,0.001,0.5,10,10,0.0,50,25


### import old study

In [ ]:
file = 'wr_param_study_01.csv'

!cp {path_src}/{file} {path_data}

In [ ]:
#df_result_old = pd.read_csv(f'{path_data}/{file}')
#df_result_old.head(5)

,min_df,n_components,n_neighbors,min_dist,min_cluster_size,min_samples,0,1,2,3,4,5,6,7,8,9
0,1,10,10,0.0,20,2,"['fruity', 'delicious fruity', 'fruity good', ...","['good value money', 'money good value', 'good...","['chilean cabernet sauvignon', 'cabernet sauvi...","['citrus tropical', 'citrus pear tropical', 'p...","['chilean wine', 'wine chile', 'chilean wine w...","['oak', 'oak oak', 'vanilla oak', 'oak vanilla...","['fruity scent', 'citrus scent', 'aromatic fru...","['håkon rekdal', 'like håkon rekdal', 'rekdal'...","['medium tannins', 'medium tannin', 'soft tann...","['easy drink good', 'drink easy', 'easy drinki..."
1,1,10,10,0.0,20,10,"['nice wine', 'good wine', 'great wine', 'exce...","['vanilla oak blackberry', 'oak blackberry van...","['nice chardonnay', 'good chardonnay', 'chardo...","['fruity', 'fruity good', 'tasty fruity', 'swe...","['sour taste', 'bitter', 'retrospect add taste...","['good value money', 'money good value', 'good...","['drink easy', 'easy drinking', 'easy drink', ...","['citrus tropical', 'citrus pear tropical', 'p...","['smooth dry', 'flavor smooth', 'dry smooth', ...","['cabernet sauvignon', 'chilean cabernet sauvi..."
2,1,10,10,0.0,20,20,"['good', 'great', 'excellent', 'bad', 'good va...","['nice chardonnay', 'chardonnay', 'good chardo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10,10,0.0,50,5,"['good wine', 'wine', 'sauvignon', 'cabernet s...","['nice chardonnay', 'chardonnay', 'good chardo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10,10,0.0,50,25,"['good', 'great', 'excellent', 'good value', '...","['nice chardonnay', 'chardonnay', 'good chardo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


drop param set studied before

In [ ]:
df = df_result_old[df_params.columns]
df_params = pd.concat([df_params, df]).drop_duplicates(keep=False)
len(df_params)

255

## Search params

In [70]:
#file_param = 'wr_param_study_240218' # reviews of 50 wines
#file_param = 'wr_param_study_240220' # add more metric such as median, std
file_param = 'test'

*shuffle param sets for early evaluation of param search*

In [71]:
import random

random.seed(42)
params = df_params.index.tolist()
random.shuffle(params)
df_params['shuffle'] = params
df_params = df_params.sort_values('shuffle').drop(columns='shuffle')
df_params.head()

,min_df,max_df,n_components,n_neighbors,min_dist,min_cluster_size,min_samples
760,0.010,1.0,10,20,0.00,50,25
125,0.001,0.5,20,20,0.05,100,100
641,0.010,0.5,20,50,0.10,20,20
779,0.010,1.0,10,20,0.10,50,50
234,0.001,0.5,50,50,0.10,20,2


*copy existing zip file if any*

In [72]:
z = f'{file_param}.zip'
!cp {path_src}/{z} .
!unzip -l {z}

cp: cannot stat '/content/drive/MyDrive/Colab Notebooks//test.zip': No such file or directory
unzip:  cannot find or open test.zip, test.zip.zip or test.zip.ZIP.


*make batch*

In [73]:
batch = 1 # starting from 1 such as 1, 2, 3, ...
num_params = 5 # number of params to search in each round

divide_df = lambda list_x, n: [list_x[i*n:(i+1)*n] for i in range(len(list_x) // n + max(0, min(1, len(list_x) % n)))]
run_idx = divide_df(df_params.index, num_params)
run_file = [f'a{i}' for i in range(1,len(run_idx)+1)]

df_params_b = df_params.loc[run_idx[batch-1]]
file_r = f'{file_param}_{run_file[batch-1]}'
file_s = f'{file_param}_score_{run_file[batch-1]}'

print(f'working {file_r} out of {len(run_idx)}\n')

df_params_b.head()

working test_a1 out of 195



,min_df,max_df,n_components,n_neighbors,min_dist,min_cluster_size,min_samples
760,0.010,1.0,10,20,0.00,50,25
125,0.001,0.5,20,20,0.05,100,100
641,0.010,0.5,20,50,0.10,20,20
779,0.010,1.0,10,20,0.10,50,50
234,0.001,0.5,50,50,0.10,20,2


*uncomment if no batch*

In [ ]:
#df_params_b = df_params

*run batch*

In [74]:
num_topics = None # num of topics to save in param search result
aspect='KeyBERT'


df_result = pd.DataFrame()
dict_score_mean = dict()
dict_score_median = dict()
dict_score_std = dict()

total = len(df_params_b)

for rec in tqdm(df_params_b.iloc[:total].itertuples(), total=total):
    idx = rec[0]
    kwargs = rec._asdict()
    kwargs.pop('Index', None)
    kwargs.update(params_base)

    tm_post = bertopic_batch(docs, **kwargs)
    topic_model = tm_post.topic_model

    # collect topics
    if topic_model is None:
        df = pd.DataFrame({0: ['ERROR']}).rename_axis('index')
        list_score = []
        #break # testing
    else:
        df = tm_post.get_topics(idx, num_topics=num_topics)

        # calc scores
        if num_topics is None:
            tid = None
        else:
            tid = range(num_topics)
        list_score = tm_post.calc_score(aspect=aspect, tid=tid)

    df_result = pd.concat([df_result, df])
    dict_score_mean[idx] = list_score[0]
    dict_score_median[idx] = list_score[1]
    dict_score_std[idx] = list_score[2]

df_result = df_params_b.join(df_result)

100%|██████████| 5/5 [02:00<00:00, 24.18s/it]


*combine scores*

In [75]:
def create_df(df, level0):
    if isinstance(df, dict):
        df = pd.DataFrame.from_dict(df, orient='index')
    df.columns = pd.MultiIndex.from_product([[level0], df.columns])
    return df

df_score = create_df(df_params_b, 'param').join(
    [create_df(dict_score_mean, 'mean'),
     create_df(dict_score_median, 'median'),
     create_df(dict_score_std, 'std')]
    )

*join param set to result if param search interupted*

In [ ]:
#df_result = df_params_b.loc[df_result.index].join(df_result)
#len(df_result)

### glimpse of results

*check if any set of error*

In [76]:
cond = df_result[0].isna()
print('num of param sets to be studied:', cond.sum())

#df_result = df_result.loc[~cond]

df_result.loc[cond].head()

num of param sets to be studied: 0


,min_df,max_df,n_components,n_neighbors,min_dist,min_cluster_size,min_samples,0,1,2,...,12,13,14,15,16,17,18,19,20,21


*topic label and score*

In [146]:
# compare topics
topic_id = 0
num_words = 5
num_params = 5

_ = [print(f'{rec["index"]:>4.0f}:', ', '.join(rec[topic_id][:num_words])) for _, rec in df_result.reset_index().iterrows() if i <= num_params]

print('\ntopic, mean, median, std')
fun = lambda rec: ', '.join([f'{rec[x][topic_id]:.3f}' for x in ['mean', 'median', 'std']])
_ = [print(f'{rec["index"][0]:>4.0f}:', f'{fun(rec)}') for _, rec in df_score.reset_index().iterrows() if i <= num_params]

 760: oak vanilla, vanilla oak, vanilla oak cherry, vanilla cherry, vanilla
 125: vanilla red fruit, vanilla oak strawberry, fruit vanilla, cherry butter strawberry, oak blackberry vanilla
 641: vanilla oak cherry, oak flavor, oak blackberry vanilla, cherry vanilla, oak cherry
 779: good, vanilla, oak vanilla, great, vanilla oak
 234: good wine, nice wine, great wine, wine good, wine great

topic, mean, median, std
 760: 0.694, 0.670, 0.107
 125: 0.605, 0.607, 0.020
 641: 0.627, 0.603, 0.046
 779: 0.565, 0.552, 0.066
 234: 0.796, 0.828, 0.087


In [89]:
#df_result.head()
df_score.head()

param                                                            \
    min_df max_df n_components n_neighbors min_dist min_cluster_size   
760  0.010    1.0           10          20     0.00               50   
125  0.001    0.5           20          20     0.05              100   
641  0.010    0.5           20          50     0.10               20   
779  0.010    1.0           10          20     0.10               50   
234  0.001    0.5           50          50     0.10               20   

                     mean                      ...       std            \
    min_samples         0         1         2  ...        12        13   
760          25  0.694203  0.761439  0.924480  ...       NaN       NaN   
125         100  0.604869  0.633347  0.754985  ...       NaN       NaN   
641          20  0.626956  0.595717  0.768470  ...       NaN       NaN   
779          50  0.564748  0.640530       NaN  ...       NaN       NaN   
234           2  0.795723  0.577253  0.924480  ...  0.032867  0.064363   

                                                                         \
           14        15        16      17        18        19        20   
760       NaN       NaN       NaN     NaN       NaN       NaN       NaN   
125       NaN       NaN       NaN     NaN       NaN       NaN       NaN   
641       NaN       NaN       NaN     NaN       NaN       NaN       NaN   
779       NaN       NaN       NaN     NaN       NaN       NaN       NaN   
234  0.038025  0.015875  0.074847  0.0375  0.022167  0.020906  0.029946   

               
           21  
760       NaN  
125       NaN  
641       NaN  
779       NaN  
234  0.021621  

[5 rows x 73 columns]

### Save

In [ ]:
f1 = f'{path_data}/{file_r}.csv'
df_result.to_csv(f1, index = False)

f2 = f'{path_data}/{file_s}.csv'
df_score.to_csv(f2, index = False)

In [ ]:
z = f'{file_param}.zip'
!zip -ju {z} {f1} {f2}
!cp {z} {path_src}

In [ ]:
!unzip -l {z}

Archive:  wr_param_study_240220.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
   863285  2024-02-20 08:55   wr_param_study_240220_a1.csv
   295784  2024-02-20 08:55   wr_param_study_240220_score_a1.csv
---------                     -------
  1159069                     2 files


## Review result

### Import result

In [4]:
file = 'wr_param_study_240220'
!unzip {path_src}/{file}.zip -d {path_data}

Archive:  /content/drive/MyDrive/Colab Notebooks//wr_param_study_240220.zip
replace sample_data/wr_param_study_240220_a1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
pstudy = param_study()

In [6]:
f = f'{file}_a'
df_result = pstudy.import_topic(f, path_data)
df_result.head()

,min_df,max_df,n_components,n_neighbors,min_dist,min_cluster_size,min_samples,0,1,2,...,85,86,87,88,89,90,91,92,93,94
0,0.010,1.0,10,20,0.00,50,25,"[oak vanilla, vanilla oak, vanilla, butter, cr...","[good wine, wine good, nice wine, great wine, ...","[good chardonnay, nice chardonnay, chardonnay,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.001,0.5,20,20,0.05,100,100,"[cherry blackberry, blackberry cherry, blackbe...","[good dessert wine, dessert wine, cabernet sau...","[chardonnay fruity, decent chardonnay, tasty c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.010,0.5,20,50,0.10,20,20,"[apple peach, tobacco, green apple, peach apri...","[good wine, wine good, wine good wine, excelle...","[chardonnay fruity, decent chardonnay, tasty c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.010,1.0,10,20,0.10,50,50,"[good, great, excellent, bad, better, nice, pl...","[good wine, excellent wine, nice wine, wine go...","[good chardonnay, nice chardonnay, excellent c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.001,0.5,50,50,0.10,20,2,"[good wine, excellent wine, wine good, nice wi...","[chardonnay, chardonnay good, nice chardonnay,...","[apple honey peach, honey peach apple, peach h...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*import score*

In [7]:
f = f'{file}_score'
pstudy.import_score(f, path_data, header=[0,1])
pstudy.df_score.head()

0 duplicated in 800 param sets


param                                                            \
  min_df max_df n_components n_neighbors min_dist min_cluster_size   
0  0.001    1.0           20          10     0.00               50   
1  0.001    0.5           50          50     0.05               20   
2  0.010    0.5           50          50     0.05               50   
3  0.010    1.0           50          20     0.10               20   
4  0.010    1.0           50          50     0.05               20   

                   mean                      ... std                          \
  min_samples         0         1         2  ...  87  88  89  90  91  92  93   
0          25  0.524725  0.859290  0.894504  ... NaN NaN NaN NaN NaN NaN NaN   
1          10  0.796091  0.881727  0.928864  ... NaN NaN NaN NaN NaN NaN NaN   
2          25  0.649124  0.834250  0.671677  ... NaN NaN NaN NaN NaN NaN NaN   
3          10  0.834960  0.928864  0.825880  ... NaN NaN NaN NaN NaN NaN NaN   
4          20  0.789649  0.895278  0.919408  ... NaN NaN NaN NaN NaN NaN NaN   

  mean median std  
    94     94  94  
0  NaN    NaN NaN  
1  NaN    NaN NaN  
2  NaN    NaN NaN  
3  NaN    NaN NaN  
4  NaN    NaN NaN  

[5 rows x 292 columns]

### Metric: c-TF-IDF of KeyBERT

*stack scores for visualization*

In [8]:
pstudy.stack()
pstudy.df_score_stacked.head()

,min_df,max_df,n_components,n_neighbors,min_dist,min_cluster_size,min_samples,Topic,mean,median,std
0,0.001,1.0,20,10,0.0,50,25,0,0.524725,0.496132,0.069180
1,0.001,1.0,20,10,0.0,50,25,1,0.859290,0.850308,0.027185
2,0.001,1.0,20,10,0.0,50,25,2,0.894504,0.898557,0.044243
3,0.001,1.0,20,10,0.0,50,25,3,0.697568,0.683256,0.152222
4,0.001,1.0,20,10,0.0,50,25,4,0.924828,0.925862,0.020236


In [9]:
#kw = ['x', 'y', 'color', 'facet_row', 'size']
#params = ['Topic', 'mean', 'n_components', 'n_neighbors', 'std']
#params = ['Topic', 'mean', 'n_neighbors', 'min_cluster_size', 'std']

kw = ['x', 'y', 'color', 'facet_row', 'error_y']
params = ['Topic', 'mean', 'n_neighbors', 'min_cluster_size', 'std']

pstudy.visualize(params, kw,
                 width=1200, height=800,
                 kwa_optional = {'marginal_y': 'box'},
                 marker_size=3
                 )

In [10]:
cols_param = list(pstudy.df_score_stacked.columns)[:7]
cols_param

['min_df',
 'max_df',
 'n_components',
 'n_neighbors',
 'min_dist',
 'min_cluster_size',
 'min_samples']

In [11]:
num_plots = 5
metric = 'mean'

kws = {
    'kw': ['y', 'x', 'color', 'facet_col', 'facet_row'],
    'kwa_starts': [metric]
}

kws = {
    'kw': ['x', 'y', 'color', 'facet_col', 'facet_row'],
    'kwa_starts': ['Topic', metric]
}

pstudy.split_param_combinations(cols_param, num_plots=num_plots, **kws)

Plot groups from 0 to 6 created.


In [12]:
figs = pstudy.visualize_all(0)

In [13]:
_ = [f.show() for f in figs]

## Topic modeling of param study result

In [ ]:
res_df = (df_result[cols_topic]
          .apply(lambda x: x.str.join(', '))
          .fillna('')
          .apply(lambda x: x.tolist(), axis=1)
          .apply(lambda row: '; '.join(filter(None, map(str, row))))
)

res_docs = res_df.tolist()
res_docs_id = res_df.index

len(res_docs)

5

In [ ]:
res_docs[0]

'wine aroma, red wine, wines, wine good, good wine, nice wine, wine, wine medium, wine oak, great wine; good chardonnay, nice chardonnay, chardonnay good, excellent chardonnay, chardonnay, chardonnay nice, delicious chardonnay, chardonnay great, best chardonnay, chardonnay chardonnay; wine good price, excellent wine price, nice wine price, value money wine, cheap wine good, wine value money, wine value, wine price, cheap wine, value wine; honey peach apple, honey apple peach, peach honey apple, peach apple honey, honey peach, honey peach green, honey green apple, peach honey, peach honey pear, honey peach honey; good dessert wine, dessert wine, dessert wine good, wine good dessert, wine dessert wine, nice dessert wine, perfect dessert wine, dessert wine nice, wine dessert, dessert wine dessert; fruity scent, fruit scent, fruity aroma, scent taste, scent scent, sweet scent, scent, scent good, scent smells, strong alcohol scent; berries fruity, fruit flavor, strawberries good, berries, f

In [ ]:
from sklearn.cluster import KMeans

#cluster_model = KMeans(n_clusters=min_cluster_size)
cluster_model = None

In [ ]:
min_dist = 0.2
min_cluster_size = 10
random_state = 42

st_id = 'all-MiniLM-L12-v2'
embedding_model = SentenceTransformer(st_id)

topic_model = bertopic_batch(res_docs,
                            min_df=0,
                            max_df=1.0,
                            n_components=15,
                            n_neighbors=10,
                            min_dist=min_dist,
                            min_cluster_size=min_cluster_size,
                            embedding_model=embedding_model,
                            calculate_probabilities=True,
                            random_state=random_state,
                            hdbscan_model=cluster_model)
topic_model.get_topic_info().head()

In [ ]:
print_topic_info(topic_model)

num of topics: 5
outliers: 0.009


In [ ]:
topic_model.visualize_topics()

In [ ]:
#topic_model.visualize_distribution(probs[0])

In [ ]:
# Larger values of min_dist will focus on the preservation of the broad topological structure.
#min_dist=0.1
#min_dist=0.9

embedding_model = SentenceTransformer(st_id)
res_embeddings = embedding_model.encode(res_docs, show_progress_bar=False)

res_reduced = UMAP(n_components=2, random_state=random_state, min_dist=min_dist).fit_transform(res_embeddings)

In [ ]:
##d = res_docs
#d = [f'{i}: {rec[0]}' for i, rec in pd.DataFrame(res_df).iterrows()]
#d = [f'{x}:{y}' for x, y in zip(res_docs_id, res_docs)]

d = [f'{x[0]}: {list(x)[1:]}' for x in df_result.iloc[:, :-10].to_records()]
title = ', '.join([x for x in list(df_result.iloc[:, :-10].columns)])

topic_model.visualize_documents(d, reduced_embeddings=res_reduced,
                                title=title,
                                hide_annotations=True)

In [ ]:
# Extract hierarchical topics and their representations
hierarchical_topics = topic_model.hierarchical_topics(res_docs)

# Visualize these representations
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 30/30 [00:00<00:00, 104.95it/s]


In [ ]:
topic_model.visualize_barchart()

#### Topics per Class
- min_df: makes no difference which could be due to its small value

In [ ]:
n = 12
visualize_topics_per_param(df_result, topic_model, res_docs, top_n_topics=n)